## Calibration

The `Calibration` class provides a way to adjust weights of observations in a dataset to match specified target values. This is commonly used in survey research and policy modeling for rebalancing datasets to better represent desired population characteristics. 

The calibration process uses an optimization algorithm to find weights that minimize the distance from the original weights while achieving the target constraints.

## Basic usage

### Parameters

`__init__(data, weights, targets)`

- `data` (pd.DataFrame): The dataset to be calibrated. This should contain all the variables you want to use for calibration.
- `weights` (np.ndarray): Initial weights for each observation in the dataset. Typically starts as an array of ones for equal weighting.
- `targets` (np.ndarray): Target values that the calibration process should achieve. These correspond to the desired weighted sums.

Calibration can be easily done by initializing the `Calibration` class, passing in the parameters above. Then `calibrate()` method performs the actual calibration using the reweight function. This method:
- Adjusts the weights to better match the target values
- May subsample the data for efficiency
- Updates both `self.weights` and `self.data` with the calibrated results

## Example

Below is a complete example showing how to calibrate a dataset to match income targets for specific age groups:

In [1]:
from microcalibrate.calibration import Calibration
import logging
import numpy as np
import pandas as pd

logging.basicConfig(
    level=logging.INFO,
)

# Create a sample dataset with age and income data
random_generator = np.random.default_rng(0)
data = pd.DataFrame({
    "age": random_generator.integers(18, 70, size=100),
    "income": random_generator.normal(40000, 50000, size=100),
})

# Set initial weights (all one in this example)
weights = np.ones(len(data))

# Calculate target values: total income for age groups 20-30 and 40-50 (as an example) or employ existing targets
targets_matrix = pd.DataFrame({
    "income_aged_20_30": ((data["age"] >= 20) & (data["age"] <= 30)).astype(float) * data["income"],
    "income_aged_40_50": ((data["age"] >= 40) & (data["age"] <= 50)).astype(float) * data["income"],
})

# 15% higher than the sum of data with the original weights
targets = np.array([
    (targets_matrix["income_aged_20_30"] * weights * 1.15).sum(), 
    (targets_matrix["income_aged_40_50"] * weights * 1.15).sum(), 
])

print(f"Original weights: {weights}")
print(f"Original targets: {targets}")

Original weights: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
Original targets: [ 245346.0788293  1237963.09787856]


In [2]:
# Initialize the Calibration object
calibrator = Calibration(
    loss_matrix=targets_matrix,
    weights=weights, 
    targets=targets,
    noise_level=0.05,
    epochs=528,
    learning_rate=0.01,
    dropout_rate=0,
    subsample_every=0,
)

# Perform the calibration
calibrator.calibrate()

print(f"Original dataset size: {len(targets_matrix)}")
print(f"Calibrated dataset size: {len(calibrator.loss_matrix)}")
print(f"Number of calibrated weights: {len(calibrator.weights)}")

INFO:microcalibrate.reweight:Starting calibration process for targets ['income_aged_20_30' 'income_aged_40_50']: [ 245346.0788293  1237963.09787856]
INFO:microcalibrate.reweight:Original weights - mean: 1.0000, std: 0.0000
INFO:microcalibrate.reweight:Initial weights after noise - mean: 1.0250, std: 0.0140
Reweighting progress:   0%|          | 0/528 [00:00<?, ?epoch/s, loss=0.0125, count_observations=100, weights_mean=1.03, weights_std=0.014, weights_min=1]INFO:microcalibrate.reweight:Within 10% from targets: 0.00% 

Reweighting progress:   0%|          | 1/528 [00:00<00:54,  9.59epoch/s, loss=0.000619, count_observations=100, weights_mean=1.05, weights_std=0.0488, weights_min=0.931]INFO:microcalibrate.reweight:Within 10% from targets: 100.00% 

INFO:microcalibrate.reweight:Epoch   10: Loss = 0.000619, Change = 0.011863 (improving)
Reweighting progress:   0%|          | 1/528 [00:00<00:54,  9.59epoch/s, loss=0.000542, count_observations=100, weights_mean=1.05, weights_std=0.063, weigh

Original dataset size: 100
Calibrated dataset size: 100
Number of calibrated weights: 100


In [3]:
# Calculate final weighted totals
final_totals = targets_matrix.mul(calibrator.weights, axis=0).sum().values

print(f"Target totals: {targets}")
print(f"Final calibrated totals: {final_totals}")
print(f"Difference: {final_totals - targets}")
print(f"Relative error: {(final_totals - targets) / targets * 100}")

Target totals: [ 245346.0788293  1237963.09787856]
Final calibrated totals: [ 245345.07223249 1237962.12413891]
Difference: [-1.00659681 -0.97373966]
Relative error: [-4.10276297e-04 -7.86565979e-05]


In [4]:
np.testing.assert_allclose(
        final_totals,
        targets,
        rtol=0.01,  # relative tolerance
        err_msg="Calibrated totals do not match target values",
    )